In [1]:
import json
import pandas as pd
import numpy as np
from google.colab import drive

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from simpletransformers.classification import MultiLabelClassificationModel
from simpletransformers.classification import ClassificationModel, ClassificationArgs


In [2]:
# !git clone https://github.com/NVIDIA/apex
# !pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [3]:
!pip install simpletransformers
#!pip install --no-cache-dir fast-bert

     |████████████████████████████████| 225kB 17.0MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
     |████████████████████████████████| 7.4MB 42.0MB/s 
     |████████████████████████████████| 1.4MB 51.6MB/s 
     |████████████████████████████████| 2.9MB 48.4MB/s 
     |████████████████████████████████| 1.8MB 44.1MB/s 
     |████████████████████████████████| 317kB 54.4MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 4.5MB 48.6MB/s 
     |████████████████████████████████| 112kB 56.9MB/s 
     |████████████████████████████████| 163kB 52.1MB/s 
     |████████████████████████████████| 102kB 14.2MB/s 
     |████████████████████████████████| 890kB 51.4MB/s 
     |████████████████████████████████| 133kB 54.1MB/s 
     |████████████████████████████████| 102kB 13.1MB/s 
     |████████████████████████████████| 122kB 55.3MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-c

In [11]:
drive.mount('/content/drive')

In [5]:
issues_df = pd.read_csv('/content/drive/My Drive/Database Project/Dataset/issue_cleaned_dataset.csv')

In [ ]:
#combining issues' body and title
#random sample

issues_df['combined']=issues_df['issue_body']+ ' | ' + issues_df['issue_title']

issues_df['combined'] = issues_df['combined'].str[:512]
issues_df['label_class'] = issues_df['label'].rank(method='dense', ascending=False).astype(int)
issues_df['label_class']=issues_df['label_class']-1


issues_df=issues_df.dropna()
issues_df=issues_df.reset_index(drop=True)

In [4]:
len(issues_df)

3892

In [ ]:
#issues_df['label_class'].value_counts()
issues_df['label_class'].value_counts()

2    1893
1    1347
0     580
Name: label_class, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(issues_df['combined'], issues_df['label_class'],
                                                   stratify=issues_df['label_class'], 
                                                    test_size=0.33)


X_train=X_train.reset_index(drop=True)
X_test=X_test.reset_index(drop=True)

y_train=y_train.reset_index(drop=True)
y_test=y_test.reset_index(drop=True)

In [ ]:
train_df=pd.DataFrame()
test_df=pd.DataFrame()

train_df['text']=X_train
train_df['labels']=y_train

test_df['text']=X_test
test_df['labels']=y_test


In [ ]:

temp=pd.DataFrame()
temp1=pd.DataFrame()
temp2=pd.DataFrame()

train_df=pd.concat([temp.append([train_df[train_df['labels']==0]]*3),
temp1.append([train_df[train_df['labels']==1]]*1),
temp2.append([train_df[train_df['labels']==2]]*1)], axis=0)
train_df['labels'].value_counts()

In [ ]:
train_df=train_df.dropna()
test_df=test_df.dropna()

In [ ]:
train_df['labels'].value_counts()

2    1268
0    1167
1     902
Name: labels, dtype: int64

In [ ]:
train_df_2=train_df[train_df['labels']==2].sample(902)
train_df_1=train_df[train_df['labels']==1].sample(902)
train_df_0=train_df[train_df['labels']==0].sample(902)

In [ ]:
train_df=pd.concat([train_df_0,train_df_1,train_df_2])

In [ ]:
train_df['labels'].value_counts()

1    902
2    902
0    902
Name: labels, dtype: int64

In [ ]:
test_df['labels'].value_counts()

2    625
1    445
0    191
Name: labels, dtype: int64

In [ ]:

#bert ,bert-large-uncased

#bert-base-uncased

model_args = ClassificationArgs(num_train_epochs=3,fp16=False,overwrite_output_dir=True)

model = ClassificationModel(
    'distilbert',
    'distilbert-base-uncased',
    num_labels=3,
    
    args=model_args
) 
#model = MultiLabelClassificationModel('distilbert', 'distilbert-base-uncased', num_labels=4, args={'train_batch_size':  4,'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 30,"save_model_every_epoch": True,"output_dir": "/content/drive/My Drive/outputs/"})


model.train_model(train_df)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(1017, 0.6205725765282484)

In [ ]:


result, model_outputs, wrong_predictions = model.eval_model(test_df)
predictions, raw_outputs = model.predict(test_df['text'])

#prediction from our test dataset

from sklearn.metrics import classification_report
import numpy as np
print(classification_report(np.array(test_df['labels'].tolist()),np.array(predictions)))

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f15c916c630>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f15c81f2198>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    sel


              precision    recall  f1-score   support

           0       0.41      0.27      0.32       191
           1       0.59      0.71      0.64       445
           2       0.85      0.81      0.83       625

    accuracy                           0.69      1261
   macro avg       0.61      0.60      0.60      1261
weighted avg       0.69      0.69      0.69      1261

